In [2]:
import pandas as pd
import dask
import dask.dataframe as dd
from elasticsearch import Elasticsearch
import nltk
import spacy
from nltk.tokenize import word_tokenize

In [3]:
doc_df = dd.read_table('msmarco-docs.tsv\msmarco-docs.tsv', blocksize=100e6,header=None)
doc_df.columns=['docid','url','title','body']
qry_df = pd.read_table('msmarco-doctrain-queries.tsv\queries.doctrain.tsv')
qry_df.columns = ['qid', 'query']
rel_df = pd.read_table('msmarco-doctrain-top100\msmarco-doctrain-top100', delimiter=' ')
rel_df.columns = ['qid', 'Q0','docid', 'rank', 'score', 'runstring']

In [4]:
temp_id = 0
temp_doc_id = []
rel = {}
for i, rec in rel_df.iterrows():
    if(rec['qid'] == temp_id):
        temp_doc_id.append(rec['docid'])
    else:
        rel[temp_id] = temp_doc_id
        temp_id = rec['qid']
        temp_doc_id = []
        temp_doc_id.append(rec['docid'])


KeyboardInterrupt: 

In [30]:
doc_df = doc_df.dropna()

In [5]:
from elasticsearch import helpers
client = Elasticsearch(
    "http://localhost:9200",
     basic_auth=("elastic", "elastic")
)
resp = client.info()

In [ ]:
for i, rec in doc_df.iterrows():
    doc = {'id': rec['docid'], 'doc': rec['body']}
    client.index(index="trec",id = i , document=doc)

In [ ]:
query = {
            "match": {
                "doc": 'what'
            }
        }
           
resp = client.search(index="trec", query=query, size=100)
print(resp)

In [6]:
all = client.search(index="trec2", query={"match_all": {}})
print("Got %d Hits:" % all['hits']['total']['value'])

Got 473 Hits:


In [43]:
nlp = spacy.load("en_core_web_sm")
class QueryProcessor:

    def __init__(self, nlp, keep=None):
        self.nlp = nlp
        self.keep = keep or {'PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'}

    def generate_query(self, text):
        doc = self.nlp(text)
        query = ' '.join(token.text for token in doc if token.pos_ in self.keep)
        return query

In [45]:
def precision(true:list, predect:list):
    f = 0
    for t in true :
        for p in predect:
            if p == t:
                f = f + 1
    return (f / len(predect))
def recall(true:list, predect:list):
    f = 0
    for t in true :
        for p in predect:
            if p == t:
                f = f + 1
    return f / len(true)

In [54]:
class noprocess:
    def __init__(self, keep=None):
        self.truth =[]
        self.predect =[]
        self.keep = keep or {'PROPN', 'NUM', 'VERB', 'NOUN', 'ADJ'}



    def removeWh(self ,q):
        query = ''
        tokens =  word_tokenize(q)
        for word, tag in nltk.pos_tag(tokens):
            print(word,tag)
            if tag in  self.keep:
                query = ' '.join(word)
        return query


    def getTruth(self, id):
        doc_list = []
        for i, rec in rel_df.iterrows():
            if id == rec['qid']:
                doc_list.append(rec['docid'])
        return doc_list

        
    def searchNoWhword(self,count):
        p = 0
        pi = 0
        r = 0
        ri = 0
        f1 = 0
        truth =[]
        predect = []
        nlp = spacy.load("en_core_web_sm")
        for i in range(1,50):
            q = QueryProcessor(nlp).generate_query(qry_df[i]['doc'])
            query = {
                        "match": {
                            "doc": q
                        }
                    }
            ids =[]
            docs =[]
            resp = client.search(index="cisi", query=query, size=count)
            for hit in resp['hits']['hits']:
                ids.append(hit['_source']['id'])
                docs.append(hit['_source']['doc'])
            predect.append(ids)
            truth.append()
            pi = precision(self.rel[str(i+1)], ids)
            p = p + pi
            ri = recall(self.rel[str(i+1)], ids)
            r = r + ri
            f1 = f1 + (2 * ((pi * ri)/(pi + ri)))
        return p / len(predect), r/len(predect), f1/len(predect)
    


    
    def searchNoSW(self,count):
        p = 0
        pi = 0
        r = 0
        ri = 0
        f1 = 0
        truth =[]
        predect = []
        for i in range(1,35):
            q = remove_stopwords(self.qry[i]['doc'] )
            query = {
                        "match": {
                            "doc": q
                        }
                    }
            ids =[]
            docs =[]
            resp = client.search(index="cisi", query=query, size=count)
            for hit in resp['hits']['hits']:
                ids.append(hit['_source']['id'])
                docs.append(hit['_source']['doc'])
            predect.append(ids)
            truth.append(self.rel[str(i+1)])
            pi = precision(self.rel[str(i+1)], ids)
            p = p + pi
            ri = recall(self.rel[str(i+1)], ids)
            r = r + ri
            f1 = f1 + (2 * ((pi * ri)/(pi + ri)))
        return p / len(predect), r/len(predect), f1/len(predect)
    

    def searchPR(self,count):
        p = 0
        pi = 0
        r = 0
        ri = 0
        f1 = 0
        truth =[]
        predect = []
        for i, rec in qry_df.iterrows():
            if i> 100:
                break
            query = {
                        "match": {
                            "doc": rec['query'] 
                        }
                    }
            ids =[]
            docs =[]
            resp = client.search(index="trec", query=query, size=count)
            for hit in resp['hits']['hits']:
                ids.append(hit['_source']['id'])
                docs.append(hit['_source']['doc'])
            predect.append(ids)
            
            truth.append(self.getTruth(rec['qid']))
            pi = precision(self.rel[str(i+1)], ids)
            p = p + pi
            ri = recall(self.rel[str(i+1)], ids)
            r = r + ri
            #print(pi,ri)
           # f1 = f1 + (2 * ((pi * ri)/(pi + ri)))
        return p / len(predect), r/len(predect) , f1/len(predect)

nop = noprocess()

In [55]:
nop.searchPR(count=100)

In [ ]:
for hit in resp['hits']['hits']:
    print(hit['_source'])
        # ids.append(hit['_source']['id'])
        # docs.append(hit['_source']['doc'])